In [58]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from datasets import load_dataset
raw_datasets = load_dataset("imdb")

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "google/electra-small-discriminator", 
    return_dict=False, 
    strict=False
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Reusing dataset imdb (/home/adamw/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/adamw/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-0943cefd6a1cd5ad.arrow
Loading cached processed dataset at /home/adamw/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-44ea5832b03319f6.arrow
Loading cached processed dataset at /home/adamw/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-ec506dbd2460bd97.arrow


In [59]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "google/electra-small-discriminator", 
    num_labels=2,
    return_dict=False
)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [60]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [61]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [62]:
import poptorch
import torch

model.electra.embeddings.position_embeddings = poptorch.BeginBlock(
    layer_to_call=model.electra.embeddings.position_embeddings,
    ipu_id=1
)

for index, layer in enumerate(model.electra.encoder.layer):
    layer = poptorch.BeginBlock(layer_to_call=layer, ipu_id=index+1)

opts = poptorch.Options().deviceIterations(8)

In [63]:
class WrappedModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        #self._model = model
        self.loss = torch.nn.CrossEntropyLoss()
        #self._loss = torch.nn.CrossEntropyLoss()

    def forward(self, input_batch, attention_batch, loss_inputs=None):
        output = self.model(input_batch, attention_batch)
        
        if loss_inputs is not None:
            loss = self.loss(output, loss_inputs)
            return output, loss
        
        return output

    def __getattr__(self, attr):
        try:
            return torch.nn.Module.__getattr__(self, attr)
        except AttributeError:
            return getattr(self.model, attr)


In [64]:
tm = poptorch.trainingModel(WrappedModel(model), options=opts)

In [65]:
from tqdm.auto import tqdm
import torch
progress_bar = tqdm(range(num_training_steps))

tm.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        #print(batch)
        input_batch = torch.tensor(batch["input_ids"])
        attention_batch = torch.tensor(batch["attention_mask"])
        batch["labels"] = batch["label"]
    
        outputs = tm(input_batch, attention_batch)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/39 [00:00<?, ?it/s]

/home/adamw/adam_env/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/home/adamw/adam_env/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Error: In poptorch/source/RemoveSurplusIdentityLosses.cpp:103: 'poptorch_cpp_error': Couldn't find a loss in graph!